# Avaliação das tendências do Saldo Migratório na Região Metropolitana de Curitiba


## Preparação do _Notebook_

Esse documento, por ser construído em Linguagem Python, apresenta textos e código, por essa razão, sua preparação depende de importação das bibliotecas que serão utilizadas nos procedimentos analíticos.

In [1]:
import pandas as pd # importação de biblioteca de manipulação de dados tabulares
import matplotlib.pyplot as plt # importação de biblioteca de plotagem de gráficos 
import numpy as np # importação de biblioteca para cálculos matemáticos
from scipy.optimize import curve_fit # importação de função a ser utilizada para ajuste de modelo preditivo 
from itertools import product # importação de função voltada para otimização de laços de iteração

### Carregamento dos Dados de residentes da RMC que moravam em outros estados em 2005

In [2]:
cod_municipios_rmc= [4100202,4100301,4100400,4101804,4102307,4103107,4104006,4104105,
                     4104204,4104253,4105201,4105805,4106209,4106902,4107652,4111258,
                     4113205,4114302,4119103,4119152,4119509,4120804,4121208,4122206,4122305,
                     4125506,4127601,4127882,4128633]

In [3]:
colunas_a_importar = [
    'Código do Item Geográfico', 'Item Geográfico',
    'Item Temporal',
    'Residência em 31/07/2005, UF', 'Residência em 31/07/2005, UF (código)',
    'Residência em 31/07/2005, UF (ordem)',
    'Frequência', 'Observações']

df_rec_migratorio_ufs = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - moradores municipios mesorregiao curitiba de acordo com UF pais de moradia em 2005.csv'
    , sep=';'
    , na_values='X'
    , usecols=colunas_a_importar

)

df_rec_migratorio_ufs = df_rec_migratorio_ufs[df_rec_migratorio_ufs['Código do Item Geográfico'].isin(cod_municipios_rmc)]

df_rec_migratorio_ufs.head(2)

,Código do Item Geográfico,Item Geográfico,Item Temporal,"Residência em 31/07/2005, UF","Residência em 31/07/2005, UF (código)","Residência em 31/07/2005, UF (ordem)",Frequência,Observações
27,4100202,ADRIANÓPOLIS,31/07/2010,"ES, Espirito Santo",32,8,NaN,NaN
28,4100202,ADRIANÓPOLIS,31/07/2010,"GO, Goias",52,9,NaN,NaN


In [4]:
df_rec_migratorio_ufs['Frequência'].sum()

107092.0

### Carregamento dos Dados de residentes da RMC que moravam em outros países em 2005

In [5]:
colunas_a_importar = [
    'Código do Item Geográfico', 'Item Geográfico',
    'Item Temporal', 'Residência em 31/07/2005, UF, município ou país estrangeiro',
    'Frequência', 'Observações']

df_rec_migratorio_extrangeiros = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - moradores municipios mesorregiao curitiba moravam em outro país em 2005.csv'
    , sep=';'
    , na_values='X'
    , usecols=colunas_a_importar

)

df_rec_migratorio_extrangeiros = df_rec_migratorio_extrangeiros[df_rec_migratorio_extrangeiros['Código do Item Geográfico'].isin(cod_municipios_rmc)]

df_rec_migratorio_extrangeiros.head(2)

,Código do Item Geográfico,Item Geográfico,Item Temporal,"Residência em 31/07/2005, UF, município ou país estrangeiro",Frequência,Observações
1,4100400,ALMIRANTE TAMANDARÉ,31/07/2010,País estrangeiro,73.0,9.0
3,4101804,ARAUCÁRIA,31/07/2010,País estrangeiro,84.0,7.0


In [6]:
df_rec_migratorio_extrangeiros['Frequência'].sum()

8252.0

### Carregamento dos Dados de residentes da RMC que moravam em outros municípios do PR em 2005

In [7]:
colunas_a_importar = [
    'Código do Item Geográfico', 'Item Geográfico',
    'Item Temporal', 'Residência em 31/07/2005, município','Residência em 31/07/2005, município (código)',
    'Frequência', 'Observações']

df_recebimento_migratorio_PR = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - moradores municipios mesorregiao curitiba de acordo com moradia em outro municipio PR em 2005.csv'
    , sep=';'
    , na_values='X'
    , usecols=colunas_a_importar

)

df_recebimento_migratorio_PR = df_recebimento_migratorio_PR[df_recebimento_migratorio_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]

df_recebimento_migratorio_PR.head(2)

,Código do Item Geográfico,Item Geográfico,Item Temporal,"Residência em 31/07/2005, município","Residência em 31/07/2005, município (código)",Frequência,Observações
328,4100202,ADRIANÓPOLIS,31/07/2010,ADRIANÓPOLIS,4100202,221.0,82.0
329,4100202,ADRIANÓPOLIS,31/07/2010,ALMIRANTE TAMANDARÉ,4100400,NaN,NaN


In [8]:
df_recebimento_migratorio_PR = df_recebimento_migratorio_PR[~df_recebimento_migratorio_PR['Residência em 31/07/2005, município (código)'].isin(cod_municipios_rmc)]

In [9]:
df_recebimento_migratorio_PR['Frequência'].sum()

54289.0

### Pessoas na RMC que moravam em outros estados, países ou municípios do Paraná exclusive aqueles da RMC

In [10]:
imigrantes_quinquenio = (df_recebimento_migratorio_PR['Frequência'].sum() + df_rec_migratorio_extrangeiros['Frequência'].sum() + df_rec_migratorio_ufs['Frequência'].sum())

print(f'Imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio)}')
print(f'Média Anual de imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio / 5)}')

Imigrantes entre 2005 e 2010: 169633
Média Anual de imigrantes entre 2005 e 2010: 33926


## Pessoas que saíram da RMC entre 2005 e 2010

In [11]:
df_perda_migratoria_PR = pd.read_csv(
    '../source/tab/originais/bme - migracao 2010 - pessoas que sairam dos 29 municípios da RMC desde 2005.csv'
    , sep=';'
    , na_values='X'
#    , usecols=colunas_a_importar

)


territorios_a_ignorar = ['ANTONINA', 'Metropolitana de Curitiba', 'GUARAQUEÇABA', 'GUARATUBA', 'MATINHOS', 'MORRETES', 'PARANAGUÁ', 'PORTO AMAZONAS']

df_perda_migratoria_PR = df_perda_migratoria_PR[~df_perda_migratoria_PR['Item Geográfico'].isin(territorios_a_ignorar)]

df_perda_migratoria_PR.head(2)

,Código do Nível Geográfico,Nível Geográfico,Código do Item Geográfico,Item Geográfico,Ordem Geográfica,Código do Item Temporal,Nível Temporal,Item Temporal,"Residência em 31/07/2005, município","Residência em 31/07/2005, município (código)","Residência em 31/07/2005, município (ordem)",Frequência,Observações
0,9000,País,76,Brasil,1,2010,8030,31/07/2010,ADRIANÓPOLIS,4100202,4100202,849.0,188.0
1,9000,País,76,Brasil,1,2010,8030,31/07/2010,AGUDOS DO SUL,4100301,4100301,900.0,120.0


### Pessoas que mudaram entre municípios da RMC, entre 2005 e 2010

In [12]:
df_perda_migratoria_PR[df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum()

177568.0

### Pessoas que saíram de Municípios da RMC (incluindo outros municípios)

In [13]:
df_perda_migratoria_PR[~df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum()

308811.0

In [14]:
Pessoas_sairam_curitiba = df_perda_migratoria_PR[~df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum() - df_perda_migratoria_PR[df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum()

In [15]:
print(f'Imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio)}')
print(f'Média Anual de imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio / 5)}')
print(f'Emigrantes entre 2005 e 2010: {int(Pessoas_sairam_curitiba)}')
print(f'Média Anual de Emigrantes entre 2005 e 2010: {int(Pessoas_sairam_curitiba / 5)}')
print(f'Saldo Migratório entre 2005 e 2010: {int(imigrantes_quinquenio - Pessoas_sairam_curitiba)}')
print(f'Saldo Migratório Médio Anual entre 2005 e 2010: {int((imigrantes_quinquenio - Pessoas_sairam_curitiba) / 5)}')


Imigrantes entre 2005 e 2010: 169633
Média Anual de imigrantes entre 2005 e 2010: 33926
Emigrantes entre 2005 e 2010: 131243
Média Anual de Emigrantes entre 2005 e 2010: 26248
Saldo Migratório entre 2005 e 2010: 38390
Saldo Migratório Médio Anual entre 2005 e 2010: 7678


In [16]:
df_perda_migratoria_PR[~df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)].head(60)

,Código do Nível Geográfico,Nível Geográfico,Código do Item Geográfico,Item Geográfico,Ordem Geográfica,Código do Item Temporal,Nível Temporal,Item Temporal,"Residência em 31/07/2005, município","Residência em 31/07/2005, município (código)","Residência em 31/07/2005, município (ordem)",Frequência,Observações
0,9000,País,76,Brasil,1,2010,8030,31/07/2010,ADRIANÓPOLIS,4100202,4100202,849.0,188.0
1,9000,País,76,Brasil,1,2010,8030,31/07/2010,AGUDOS DO SUL,4100301,4100301,900.0,120.0
2,9000,País,76,Brasil,1,2010,8030,31/07/2010,ALMIRANTE TAMANDARÉ,4100400,4100400,5852.0,588.0
3,9000,País,76,Brasil,1,2010,8030,31/07/2010,ARAUCÁRIA,4101804,4101804,7565.0,838.0
4,9000,País,76,Brasil,1,2010,8030,31/07/2010,BALSA NOVA,4102307,4102307,682.0,82.0
5,9000,País,76,Brasil,1,2010,8030,31/07/2010,BOCAIÚVA DO SUL,4103107,4103107,796.0,128.0
6,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPINA GRANDE DO SUL,4104006,4104006,3198.0,405.0
7,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPO DO TENENTE,4104105,4104105,695.0,116.0
8,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPO LARGO,4104204,4104204,7132.0,833.0
9,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPO MAGRO,4104253,4104253,1758.0,175.0


### Emigração Internacional 2005 a 2010

In [52]:
df_perda_migratoria_internacional = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - pessoas que sairam dos 29 municípios da RMC para outro pais 2005 a 2010 sem faixas etárias.csv'
    , sep=';'
    ,na_values='X'
)

df_perda_migratoria_internacional[
    (~df_perda_migratoria_internacional['Código do Item Geográfico'].isin(cod_municipios_rmc)) 
    & 
    (df_perda_migratoria_internacional['Item Geográfico'] != 'Metropolitana de Curitiba')
    
    ].groupby(by='Emigrante, sexo')['Frequência'].sum()


# Cálculo Emigrantes anuais da RMC entre 2005 e 2010, subtraindo os dados da Mesorregião Metropolitana
((df_perda_migratoria_internacional[
    (df_perda_migratoria_internacional['Item Geográfico'] == 'Metropolitana de Curitiba')
    ].groupby(by='Emigrante, sexo')['Frequência'].sum() ## emigrantes, agrupados por sexo e de toda Mesorregião
    -
df_perda_migratoria_internacional[
    (~df_perda_migratoria_internacional['Código do Item Geográfico'].isin(cod_municipios_rmc)) 
    & 
    (df_perda_migratoria_internacional['Item Geográfico'] != 'Metropolitana de Curitiba')## emigrantes, agrupados por sexo do litoral
    
    ].groupby(by='Emigrante, sexo')['Frequência'].sum()) /  5.5 #agrupamento por sexo e divisão por 5,5 anos
).astype(int)

Emigrante, sexo
Feminino     947
Masculino    896
Name: Frequência, dtype: int64